In [1]:
import torch
import sys
sys.path.append('..')
from model.utils import LMHyperParams, SmModel, ModelChoice
from dataset.squad import UltraFeedbackDataModule
from transformers import AutoTokenizer, PreTrainedTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft.tuners.lora.config import LoraConfig
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id

tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(model_id) # type: ignore
data_module = UltraFeedbackDataModule(2, tokenizer, 1024, 10)
data_module.setup("fit")

2024-11-23 17:39:38.958 | INFO     | dataset.squad:setup:219 - Loading dataset for stage fit
2024-11-23 17:39:40.881 | INFO     | dataset.squad:setup:230 - Processing dataset for stage fit, workers: 12, cache dir dataset_caches/ultrafeedback
num_proc must be <= 9. Reducing num_proc to 9 for dataset of size 9.
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


In [ ]:
model_id = "cognitivecomputations/dolphin-2.1-mistral-7b" # replace with your model id

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    use_cache=False, 
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' # to prevent errors with FA
tokenizer.truncation_side = 'left' # to prevent cutting off last generation

Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.24s/it]


In [5]:
prompt_length = 1024
max_seq_length = 1512

peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.05,
    r=256,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

args = DPOConfig(
    output_dir="doplhin-dpo",
    num_train_epochs=1,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=5e-5,
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_steps=25,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=700,
    bf16=True,
    tf32=True,
    push_to_hub=False,
    report_to="tensorboard",
)

dpo_args = {
    "beta": 0.1,  # The beta factor in DPO loss. Higher beta means less divergence
    "loss_type": "sigmoid",  # The loss type for DPO.
}


trainer = DPOTrainer(
    model,
    ref_model=None,  # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=data_module.train_dataset,
    eval_dataset=data_module.val_dataset,
    tokenizer=tokenizer,  # type: ignore
    max_length=max_seq_length,
    max_prompt_length=prompt_length,
    beta=dpo_args["beta"],
    loss_type=dpo_args["loss_type"],
)

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_length, max_prompt_length, loss_type. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:475: UserWarning: 

In [11]:
dataloader = data_module.train_dataloader()
first_batch = next(iter(dataloader))
trainer.compute_loss(model, first_batch, "train")

KeyError: 'prompt_input_ids'

In [6]:
trainer.train()

/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_input_ids = [torch.tensor(example["prompt_input_ids"]) for example in examples]
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  chosen_input_ids = [torch.tensor(example["chosen_input_ids"]) for example in examples]
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/trl/trainer/dpo_trainer.py:126: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requi

Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=0.6931471824645996, metrics={'train_runtime': 20.0123, 'train_samples_per_second': 0.45, 'train_steps_per_second': 0.05, 'total_flos': 0.0, 'train_loss': 0.6931471824645996, 'epoch': 1.0})